In [1]:
import tensorflow as tf
from tensorflow import keras
from gym.spaces import Box
import numpy as np
from EnvWrapperRay import HungryGeeseKaggle

Loading environment football failed: No module named 'gfootball'


In [2]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
from kaggle_environments import make
env = make("hungry_geese")

In [3]:
env.state[0].observation

{'remainingOverageTime': 60,
 'step': 0,
 'geese': [[51]],
 'food': [41, 4],
 'index': 0}

In [4]:
import ray.rllib.agents.ppo as ppo
import ray

In [5]:
ray.shutdown()

In [6]:
ray.init(log_to_driver=False)

2021-06-08 11:32:22,345	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.120',
 'raylet_ip_address': '192.168.1.120',
 'redis_address': '192.168.1.120:6379',
 'object_store_address': '/tmp/ray/session_2021-06-08_11-32-21_369153_10639/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-08_11-32-21_369153_10639/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-06-08_11-32-21_369153_10639',
 'metrics_export_port': 57830,
 'node_id': '1574a80bc906ca2dfede59623d65497304a36c5fbd67425692b7aa4e'}

In [7]:
tf.compat.v1.disable_eager_execution()

In [151]:
path = '../tmp/ppo-lstm/geese/checkpoint_000102/checkpoint-102'
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
config['model'] = {
#           "custom_model": "my_model"
        "conv_filters": [[3,4,8]],
        "use_lstm": True,
        "lstm_use_prev_action": True,
        "poo": [256,256],
        "post_fcnet_activation": "relu",
      }
#config['conv_filters'] = [[32,16,1]]

#config['Q_model'] = {
#           "custom_model": "my_model"
#        }
config["no_done_at_end"] = True
config["normalize_actions"] =  False

agent = ppo.PPOTrainer(config, env=HungryGeeseKaggle)
agent.restore(path)

#policy = agent.get_policy()

2021-06-08 14:28:20,533	INFO trainer.py:669 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-06-08 14:28:20,533	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-06-08 14:28:20,586	ERROR actor.py:702 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=10639, ip=192.168.1.120)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
  File "/home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 514, in __init__
    

RayTaskError(AttributeError): [36mray::RolloutWorker.foreach_policy()::Exiting[39m (pid=10639, ip=192.168.1.120)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
  File "/home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "/home/rhys/Projects/RLxEvolution/RLxEvolution/lib/python3.8/site-packages/ray/rllib/evaluation/rollout_worker.py", line 995, in foreach_policy
    for pid, policy in self.policy_map.items()
AttributeError: 'NoneType' object has no attribute 'items'

In [9]:
config_env = {
    'rows': 11,
    'columns': 7
}

In [10]:
env_wrapper = HungryGeeseKaggle(config_env)
env_wrapper.reset()
obs = env_wrapper.get_geese_observation(0, env.state)

In [11]:
state=[np.zeros(256, np.float32),
               np.zeros(256, np.float32)]

In [12]:
action = agent.compute_action(obs, state, prev_action=5)[0]
state = agent.compute_action(obs, state, prev_action=1)[1]

In [34]:
ray.shutdown()

In [163]:
ray.shutdown()
import importlib
import main
importlib.reload(main)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq_in (InputLayer)             [(None,)]            0                                            
__________________________________________________________________________________________________
tf_op_layer_default_policy/Sequ [()]                 0           seq_in[0][0]                     
__________________________________________________________________________________________________
tf_op_layer_default_policy/Sequ [()]                 0           tf_op_layer_default_policy/Sequen
__________________________________________________________________________________________________
tf_op_layer_default_policy/Sequ [(None, 1)]          0           seq_in[0][0]                     
____________________________________________________________________________________________

<module 'main' from '/home/rhys/Projects/RLxEvolution/submission_ray/main.py'>

In [28]:
env_wrapper.env.state[0].observation

{'remainingOverageTime': 60,
 'step': 0,
 'geese': [[57], [10], [65], [71]],
 'food': [60, 4],
 'index': 0}

In [161]:
state = env_wrapper.reset()

In [106]:
#ray_info = ray.init(configure_logging=True, logging_level='ERROR')

In [162]:
main.agent(env_wrapper.env.state[0].observation, config_env)

{'remainingOverageTime': 60, 'step': 0, 'geese': [[35], [4], [61], [24]], 'food': [20, 60], 'index': 0}
last action: 1


'NORTH'